# Using the DataLoader

In [1]:
# Import code from file in upper directory
import sys, os
sys.path.append(os.getcwd() + os.sep + os.pardir)
from tweet_data import TweetsBaseDataset

Let's load the dev dataset for simplicity.

In [2]:
dataset = TweetsBaseDataset('../data/dev', 'us_trial')

Reading file
Read file with 50000 tweets, 77053 unique tokens
Building vocabulary
Loading labels


To be able to get batches of variable-length sequences we have to write our custom `collate_fn` for the `DataLoader`. This function is defined in `TweetBaseDataset.collate_fn()`.

In [3]:
from torch.utils.data import DataLoader
data_loader = DataLoader(dataset, collate_fn=TweetsBaseDataset.collate_fn, batch_size=4, shuffle=True)

We can now use the data loader to get batches of the data. Each batch contains the padded sequences, the labels, and the length of each sequence. Padding is inserted with zeros (consistent with `dataset.vocabulary['<PAD>']`, which maps to 0) and sequences are sorted from longest to shortest:

In [4]:
data, labels, lengths = next(iter(data_loader))
print('Padded sequences:\n', data)
print('Labels:\n', labels)
print('Sequence lenghts:\n', lengths)

Padded sequences:
 tensor([[  73,    1,  240,   74],
        [  16,   57,   18,    1],
        [ 176,  192,    1,    3],
        [ 167,    1,    1,   22],
        [  12,   10,    5,    1],
        [  42,  625,  871,    0],
        [  15,  149,   17,    0],
        [   9,  711,    7,    0],
        [ 438,  235, 5753,    0],
        [ 187,  133,    3,    0],
        [  81, 8899, 6441,    0],
        [   1,    1, 3219,    0],
        [  69,    3, 1517,    0],
        [ 378, 1996,    1,    0],
        [   4,    5,    0,    0],
        [   3,  662,    0,    0],
        [   1,    0,    0,    0],
        [   5,    0,    0,    0],
        [ 273,    0,    0,    0]])
Labels:
 tensor([ 0, 10,  1,  1])
Sequence lenghts:
 [19 16 14  5]


The sequence lengths can be used to create a `PackedSequence`, which avoids calculating the output of recurrent models for padding tokens. A `PackedSequence` is created using `pack_padded_sequence()`:

In [5]:
import torch
from torch.nn.utils.rnn import pack_padded_sequence

# Model definition
embedding_dim = 100
embeddings = torch.nn.Embedding(len(dataset.vocabulary), embedding_dim)
rnn = torch.nn.RNN(embedding_dim, embedding_dim)
linear = torch.nn.Linear(embedding_dim, 6)

# Forward pass with padded batch of data
def example_forward(data, lengths):
    x = embeddings(data)
    x = pack_padded_sequence(x, lengths)
    _, x = rnn(x)
    x = linear(x)
    
    return x

print(example_forward(data, lengths))

tensor([[[ 0.2585, -0.1482, -0.2173,  0.2026, -0.1283, -0.0888],
         [ 0.3145,  0.4838, -0.2319,  0.3397,  0.0161,  0.0036],
         [ 0.3235,  0.0585,  0.2628,  0.2954,  0.1079,  0.0250],
         [ 0.3393, -0.0997, -0.0520,  0.1779,  0.3314, -0.0976]]],
       grad_fn=<ThAddBackward>)


Note that throughout these examples we have been using the default setting in PyTorch where the first axis corresponds to the sequence, and the second corresponds to batches.